In [ ]:
# Cell 1: Import necessary libraries
import numpy as np
import os
import time
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser, RigidTransform, RotationMatrix,
    Role, MeshcatVisualizer, StartMeshcat, RationalForwardKinematics, CspaceFreePolytope,
    SeparatingPlaneOrder, Rgba, InverseKinematics, RollPitchYaw,
    LinearEqualityConstraint, Sphere, Parallelism, AddDefaultVisualization, 
    ConnectPlanarSceneGraphVisualizer, IrisFromCliqueCoverOptions, 
    IrisInConfigurationSpaceFromCliqueCover, RandomGenerator, RobotDiagramBuilder, 
    SceneGraphCollisionChecker, MultibodyPlant, SceneGraph, 
    SolverOptions, CommonSolverOption, GeometrySet, ScsSolver
)
from pydrake.geometry.optimization import GraphOfConvexSetsOptions, HPolyhedron, VPolytope, Point, Hyperellipsoid
from pydrake.geometry.optimization import ConvexHull as DrakeConvexHull
from pydrake.planning import GcsTrajectoryOptimization
from pydrake.solvers import MathematicalProgram, Solve, MosekSolver
from pydrake.trajectories import CompositeTrajectory
from pydrake.common import FindResourceOrThrow
from scipy.spatial import ConvexHull
import mcubes
from functools import partial
import matplotlib.pyplot as plt
from ipywidgets import widgets
import quadprog

from pathlib import Path
import sys

# add the parent directory of this notebook to the import path
parent = Path.cwd().parent
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

from ciris_plant_visualizer import CIrisPlantVisualizer

In [ ]:
# Cell 2: Set up the plant and scene graph, and initialize the CIrisPlantVisualizer
# builder = DiagramBuilder()
# plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
# parser = Parser(plant, scene_graph)
# parser.SetAutoRenaming(True)

print("Setting up the plant and scene graph...")

# Replace DiagramBuilder with RobotDiagramBuilder
builder = RobotDiagramBuilder(time_step=0.0)
plant = builder.plant()
scene_graph = builder.scene_graph()
parser = Parser(plant, scene_graph)
parser.SetAutoRenaming(True)

In [ ]:
# Add the robot
# gripper = parser.AddModels(file_name="../my_sdfs/wsg_2dof.sdf")[0]

print("Loading Panda robot models...")

# --- Add the Panda arm + hand ---
panda_arm  = parser.AddModels(url="package://drake_models/franka_description/urdf/panda_arm.urdf")[0]
panda_hand = parser.AddModels(url="package://drake_models/franka_description/urdf/panda_hand.urdf")[0]

print("Welding Panda arm and hand...")

# Weld arm base to world (identity)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("panda_link0", panda_arm),
    RigidTransform())

# Weld hand to arm flange with X_PC: translation [0,0,0], RPY deg [0,0,-45]
X_8H = RigidTransform(RollPitchYaw(0.0, 0.0, -np.deg2rad(45.0)), [0.0, 0.0, 0.0])
plant.WeldFrames(
    plant.GetFrameByName("panda_link8", panda_arm),        # parent (P)
    plant.GetFrameByName("panda_hand", panda_hand),    # child  (C)
    X_8H)

print("Setting default finger joint positions...")

# Optional: set the default finger opening (each finger is a prismatic joint).
# 0.02 m on each finger → ~0.04 m total width. Adjust to taste.
for j in ["panda_finger_joint1", "panda_finger_joint2"]:
    plant.GetJointByName(j, panda_hand).set_default_translation(0.02)

In [ ]:
print("Adding the bottle cap and obstacles...")

cap = parser.AddModels(file_name="../my_sdfs/bottle_cap.sdf")[0]
# obstacle1 = parser.AddModels("../my_sdfs/obstacle.sdf")[0]
# # obstacle2 = parser.AddModels("my_sdfs/obstacle.sdf")[0]
# obstacle3 = parser.AddModels("../my_sdfs/obstacle.sdf")[0]

# Set welds
plant.WeldFrames(
    plant.world_frame(), 
    plant.GetFrameByName("base_link", cap),
    RigidTransform(RotationMatrix(), [0.5, 0, 0]))

# # Weld the obstacle to the world frame (adjust pose as needed)
# obstacle_pose1 = RigidTransform(RotationMatrix(), [0.51, 0.035, 0.02])  # Adjust position
# plant.WeldFrames(
#     plant.world_frame(),
#     plant.GetFrameByName("obstacle_link", obstacle1),
#     obstacle_pose1)

# # obstacle_pose2 = RigidTransform(RotationMatrix(), [-0.025, 0.05, 0.02])  # Adjust position
# # plant.WeldFrames(
# #     plant.world_frame(),
# #     plant.GetFrameByName("obstacle_link", obstacle2),
# #     obstacle_pose2)

# obstacle_pose3 = RigidTransform(RotationMatrix(), [0.465, -0.005, 0.02])  # Adjust position
# plant.WeldFrames(
#     plant.world_frame(),
#     plant.GetFrameByName("obstacle_link", obstacle3),
#     obstacle_pose3)

In [ ]:
print("Finalising plant...")
print("Bodies:", plant.num_bodies(), "Joints:", plant.num_joints())


plant.Finalize()

inspector = scene_graph.model_inspector()
num_frames = len(list(inspector.GetAllFrameIds()))
num_geoms = sum(inspector.NumGeometriesForFrame(fid) for fid in inspector.GetAllFrameIds())
print("Frames:", num_frames, "Geometries:", num_geoms)


print("Plant finalised.")

print("Number of positions: ", plant.num_positions())

# Cell 3: Initialize the CIrisPlantVisualizer
q_star = np.zeros(plant.num_positions())


print("Initialising CspaceFreePolytope...")

# The object we will use to perform our certification
cspace_free_polytope = CspaceFreePolytope(
    plant, 
    scene_graph,
    SeparatingPlaneOrder.kAffine,
    q_star)

print("Initializing CIrisPlantVisualizer...")

visualizer = CIrisPlantVisualizer(
    plant,
    builder,
    scene_graph,
    cspace_free_polytope,
    viz_role=Role.kIllustration,
    allow_plus_3dof=True
)

print("Setting up the visualizer...")

visualizer.task_space_diagram.ForcedPublish(visualizer.task_space_diagram_context)


In [ ]:
sliders = []

plant_context = visualizer.plant_context
diagram = visualizer.task_space_diagram
diagram_context = visualizer.task_space_diagram_context

for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]
    q_high = plant.GetPositionUpperLimits()[i]
    step = (q_high - q_low) / 100
    sliders.append(widgets.FloatSlider(
        min=q_low, max=q_high, 
        value=0, step=step, 
        description=f"q{i}"))
    
q = np.zeros(plant.num_positions())
# q = [-3.14, -0.78, -0.03]
def handle_slider_change(change, idx):
    q[idx] = change['new']
    # print(visualizer.check_collision_q_by_ik(q))
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)